In [9]:
import pandas as pd
import os
import sys
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the API key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
EMBED_DIM = 3072

In [10]:
df = pd.read_csv('supportx.csv')

In [12]:
df = df.drop(columns=['Unnamed: 0'])

In [13]:
df.head()

,title,link,content
0,Đăng ký thành viên Hasaki như thế nào?,https://hotro.hasaki.vn/tai-khoan.html,Quý khách vui lòng nhấn vào nút “Đăng nhập/Đăn...
1,Tại sao tôi không thể đăng nhập vào tài khoản ...,https://hotro.hasaki.vn/tai-khoan.html,"Quý khách cần kiểm tra kiểu gõ, phím Caps Lock..."
2,Tôi muốn thay đổi thông tin tài khoản thành vi...,https://hotro.hasaki.vn/tai-khoan.html,"Để thay đổi thông tin cá nhân, quý khách cần đ..."
3,Tôi có thể sử dụng chung tài khoản với người k...,https://hotro.hasaki.vn/tai-khoan.html,Không nên sử dụng chung tài khoản vì điều này ...
4,Đăng ký thành viên tại Hasaki.vn sẽ giúp ích g...,https://hotro.hasaki.vn/tai-khoan.html,Việc đăng ký tài khoản giúp khách hàng trở thà...


In [15]:
from pinecone_connection import PineConeDB

hasaki_index = PineConeDB(PINECONE_API_KEY).pc.Index('hasaki-index-v3')

In [16]:
from pc_utils import create_vector_emb

In [17]:
df['vector'] = df['title'].apply(lambda text: create_vector_emb(text))

In [18]:
df.head()

,title,link,content,vector
0,Đăng ký thành viên Hasaki như thế nào?,https://hotro.hasaki.vn/tai-khoan.html,Quý khách vui lòng nhấn vào nút “Đăng nhập/Đăn...,"[-0.008064837, 0.0011870052, 0.0071918024, 0.0..."
1,Tại sao tôi không thể đăng nhập vào tài khoản ...,https://hotro.hasaki.vn/tai-khoan.html,"Quý khách cần kiểm tra kiểu gõ, phím Caps Lock...","[-0.010309301, -0.018046966, 0.013814009, -0.0..."
2,Tôi muốn thay đổi thông tin tài khoản thành vi...,https://hotro.hasaki.vn/tai-khoan.html,"Để thay đổi thông tin cá nhân, quý khách cần đ...","[-0.024542063, -0.006162367, 0.003487309, -0.0..."
3,Tôi có thể sử dụng chung tài khoản với người k...,https://hotro.hasaki.vn/tai-khoan.html,Không nên sử dụng chung tài khoản vì điều này ...,"[-0.0393322, -0.01782562, -0.0015396316, -0.01..."
4,Đăng ký thành viên tại Hasaki.vn sẽ giúp ích g...,https://hotro.hasaki.vn/tai-khoan.html,Việc đăng ký tài khoản giúp khách hàng trở thà...,"[0.0041135163, -0.03107662, 0.0018056638, -0.0..."


In [19]:
upsert_data = []
for idx, row in df.iterrows():
    vector = row['vector']
    metadata = {
        "title": row['title'],
        "link": row['link'],
        "content": row['content']
    }
    upsert_data.append((str(idx), vector, metadata))

In [20]:
hasaki_index.upsert(vectors=upsert_data, namespace='support-namespace')

{'upserted_count': 37}